# Exploring OpenAI API directly

In [2]:
%load_ext autoreload
%autoreload 2

In [66]:
import os
import aiohttp
import json
import re
import os
import llm_notes
from pathlib import Path
import asyncio
import llm_notes
import yaml
import networkx as nx

api_key = os.environ.get('OPENAI_API_KEY')


## Barebones API calls

In [4]:
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {api_key}',
}

data = {
  "model": "text-davinci-003",
  "prompt": "Say this is a test",
  "max_tokens": 200,
  "temperature": 0,
}


async with aiohttp.ClientSession() as session:
    async with session.post(
        'https://api.openai.com/v1/completions',
        headers=headers,
        data=json.dumps(data),
    ) as response:
        response_data = await response.json()
response_data

{'id': 'cmpl-7Wixfp28Ij8RcqRvG4wTrYc5y6O1i',
 'object': 'text_completion',
 'created': 1688033683,
 'model': 'text-davinci-003',
 'choices': [{'text': '\n\nThis is indeed a test.',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 5, 'completion_tokens': 8, 'total_tokens': 13}}

In [5]:
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {api_key}',
}

data = {
  "model": "gpt-3.5-turbo",
  "messages": [{"role":"user","content":"Say this is a test"}],
  "max_tokens": 20,
  "temperature": 0,
}


async with aiohttp.ClientSession() as session:
    async with session.post(
        'https://api.openai.com/v1/chat/completions',
        headers=headers,
        data=json.dumps(data),
    ) as response:
        response_data = await response.json()
response_data

{'id': 'chatcmpl-7WixhI75HXYHy1Sf0HqVpysmTPEFr',
 'object': 'chat.completion',
 'created': 1688033685,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant', 'content': 'This is a test.'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 12, 'completion_tokens': 5, 'total_tokens': 17}}

## Simple tagger

In [6]:
async def simple_tagger(text, possible_tags, **kwargs):

    # Set up the prompt with input variables for tools, user input and a scratchpad for the model to record its workings
    template = """Tag the given text with tags from the possible tags.

    Possible Tags:
    {possible_tags}

    Return only the most relevant tags, in order of relevance.
    Return at most 3 tags, but fewer if there are fewer than 3 relevant tags.


    Return answers in the following format:
    [tag1, tag2, tag3]

    if no tags are found, return an empty list 
    []
    
    Text will be delivered in the following format:
    Text: 'Example text to be tagged'
    
    Answer with only the list of tags or an empty list
    """
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {api_key}',
    }

    data = {
    "model": "gpt-3.5-turbo",
    "messages": [
        {"role":"user","content":template.format(possible_tags=possible_tags)},
        {"role":"user","content":text}
    ],
    "max_tokens": 100,
    "temperature": 0,
    }
    data.update(kwargs)

    async with aiohttp.ClientSession() as session:
        async with session.post(
            'https://api.openai.com/v1/chat/completions',
            headers=headers,
            data=json.dumps(data),
        ) as response:
            response_data = await response.json()
    # print(response_data)
    response_text = response_data["choices"][0]["message"]["content"]
    tags = [] if response_text == "[]" else re.sub("\[|\]|'|\"", "", response_text).split(", ")
    return tags


Here is a table of the ways that a simple zero shot classification can be improved based on:
Large Language Models in the Workplac - A Case Study on Prompt Engineering for Job Type Classification

| Name | Description |
| --- | --- |
| Baseline | Provide a a job posting and asking if it is fit for a graduate. |
| CoT | Give a few examples of accurate classification before querying. |
| Zero-CoT | Ask the model to reason step-by-step before providing its answer. |
| rawinst | Give instructions about its role and the task by adding to the user msg. |
| sysinst | Give instructions about its role and the task as a system msg. |
| bothinst | Split instructions with role as a system msg and task as a user msg. |
| mock | Give task instructions by mocking a discussion where it acknowledges them. |
| reit | Reinforce key elements in the instructions by repeating them. |
| strict | Ask the model to answer by strictly following a given template. |
| loose | Ask for just the final answer to be given following a given template. |
| right | Asking the model to reach the right conclusion. |
| info | Provide additional information to address common reasoning failures. |
| name | Give the model a name by which we refer to it in conversation. |
| pos | Provide the model with positive feedback before querying it. |


Their **best prompt** which is Baseline  + bothinst + mock + reit + right + info + name+pos

In [7]:
await simple_tagger("My dog is a good boy", ["cooking", "animals"])

['animals']

## Set up note collection and graph

In [75]:
notes = llm_notes.note.load_note_directory(Path("./example_notes/"))

In [76]:
n = notes[3]
print(n)

self.title='relational note taking.md', self.tags=['notetaking'], self.sources=['https://thesephist.com/posts/inc/']


In [77]:
from pprint import pprint
pprint(n.metadata)

NoteMetadata(title='relational note taking.md',
             tags=['#notetaking'],
             note_links=['My Note taking principles',
                         'Zettelkasten',
                         'Evergreen Notes',
                         'My Note taking principles',
                         'General Zettelkasten Principles',
                         'Evergreen Notes',
                         'Obsidian',
                         'Roam research',
                         'Notion'],
             back_links=None,
             short_title='rltnl_note_tkng.')


In [78]:
note_graph = llm_notes.note.construct_note_graph(notes)

In [79]:
note_graph.nodes

NodeView(('ldd_qstns_dtls.', 'give_vs_take_cnvrs_styls_cnvrs_drknb', 'imprv_thtre_stryt_frmwr', 'rltnl_note_tkng.', 'sktch_imprv_is_wrkng_bckwr_build_lrgr_wrldv', 'obsdn', 'mnngf_spllc_actn_ecnmy', 'story_rspns_plyrs_over_just_chrct_rspns', 'gcn_graph_cnvlt_lyrs.', 'obsdn_wbste_mkdcs', 'what_is_zttlk', 'graph_nrl_ntwrk_(gnn)_intrd', 'gssn_blf_prpgt', 'how_nmss_systm_crts_strs.', 'cntrl_party_fgre_shrd_npc.m', 'gntle_intrd_graph_nrl_ntwrk', 'fctr_graph_used_prbbl_infrn', 'dstl_-_pb.md', 'death_mvs.m', 'chrch_intrr', 'gnn_2020_rvw_into_2021.', 'blnce_btwn_imprv_prior_scene_rlply', 'prprn_imprv_rpgs.', 'rtls_play', 'frtfl_void_-_rich_never_exhst_dtl', 'ldd_qstns_cmbt', 'ldd_qstns_chrct_crtn', 'swrds_wtht_mstr', 'ldd_qstns_dtls', 'ldd_sttmn_dtls', 'prprn_imprv_rpgs', 'how_nmss_systm_crts_strs', 'my_note_tkng_prncp', 'zttlk', 'evrgr_notes', 'gnrl_zttlk_prncp', 'roam_rsrch', 'ntn', 'mchnc_allow_mnngf_agncy_rpgs', 'osr', 'great_mgcs_rqre_scrfc', 'doubt_advnt_crvs_arln', 'grgna_mines_dwly', 'h

### Simple tagger

In [ ]:
all_tags = set([])
for n in notes:
    all_tags.update(n.tags)

all_tags = {
 'bayesian',
 'cite_note',
 'deeplearning',
 'gamedesign',
 'graph',
 'homebrew',
 'list',
 'mythoughts',
 'notetaking',
 'pbta',
 'pluginrules',
 'probability',
 'resource',
 'review',
 'rpgs',
 'rules',
 'stub',
 'thoughtcollection',
 'thoughts',
 'tool',
 'tutorial',
 'writing'
 }

In [55]:
print(await simple_tagger(n.body, all_tags), n.tags)

['notetaking', 'thoughtcollection', 'mythoughts'] ['notetaking']


In [56]:
pred_tags = await asyncio.gather(*[simple_tagger(n.body, ["rpgs"]) for n in notes])

KeyError: 'choices'

In [15]:
for pt, n in zip(pred_tags, notes):
    print(n.title)
    print(pt, n.tags)

Loaded Questions with details.md
['rpgs'] ['rpgs', 'thoughtcollection', 'pbta']
give vs take conversation styles and conversational doorknobs.md
['rpgs'] []
improv theatre as a storytelling framework.md
['rpgs'] ['list']
relational note taking.md
['note taking', 'incremental note taking', 'time'] ['notetaking']
Sketch improv is working backward to build a larger worldview.md
['rpgs'] ['rpg']
Obsidian.md
[] ['notetaking', 'tool']
meaningful spellcasting and the action economy.md
['rpgs'] ['rpg', 'thoughts']
Story responsibility for players over just Character responsibility.md
['rpgs'] ['rpg', 'thoughtcollection', 'pbta', 'ksbd', 'swordswithoutmaster']
GCN graph convolution layers.md
[] []
obsidian website with mkdocs.md
['rpgs'] ['writing', 'tutorial']
What is zettelkasten.md
['zettelkasten', 'Niklas Luhmann', 'systems theory'] ['thoughtcollection', 'notetaking', 'cite_note', 'cite_note', 'cite_note', 'cite_note']
Graph neural network (GNN) Introduction.md
['message passing algorithm',

In [16]:
print(notes[-2])

self.title='Balance between improv and prior scene in roleplay.md', self.tags=['rpg', 'anticannon', 'improv'], self.sources=[]


In [17]:
print(notes[-2].body)



[[Sketch improv is working backward to build a larger worldview]], starting with a detail framed with open ended connection to other unknown details. In improv you take that as writ, and then continue to flesh out and build on those connections. The process of doing this backwards is that you build scene from successively widening the lens for details. A cohesive scene is thus emergent rather than prescriptive where the details would come out of a broad framing.

In an rpg things are a little bit more flexible, in that we can, and often want to, spend some more time before hand to build details. We can also change details after the fact (retcon) to better suit a new idea or narrative as we see it evolving.

I should say ideally, because if we are to attached to things that are already established, then we cant be free with where things are going. And if we are too attached to where things are going, then we might have to run roughshod over what has been established. See [[The secret 

In [21]:
notes[0].metadata

In [19]:
print("".join([
    "---\n",
    yaml.dump(metadata),
    "---\n\n",
    "\n".join(n.lines),
]))

NameError: name 'metadata' is not defined

In [ ]:
print(n.title)
print(n.full_text)

relational note taking.md
- source: https://thesephist.com/posts/inc/
- tags: #notetaking 
Blog post codefying their note taking style, principles and tooling. informs much of [[My Note taking principles]].

Uses an iceberg analogy, where most note taking accounts for the 10% of structured, easily codefiable notes such as meeting or lecture notes, or notes from reading. But misses the 90% of thoughts that are the majority of our thought process.

Has a strong emphasis on the incremental nature of thought, and the nature of notes over time. My own past positive experience with note taking, and using those notes, was a chronological pen-and-paper lab notebook. a chronological view, with some labeling to spur thought, was the most effective way in which I actually used and referred back to my notes. This was acheived with a paper note taking system, which in this blog they discuss as being a low barrier way to capture the 90% of thought in their iceberg analogy.

Things like [[Zettelkaste